In [1]:
from vis import *
from landmarks import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Overlapped visualisation
landmarks = ["right bony anterior", 'right top orbit', "nasion"]
annotated = [3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 17, 18, 20, 21, 22, 23, 24]
meshes = []
for i in annotated:
    path = get_ply_path(i)
    mesh = o3d.io.read_triangle_mesh(path)
    mesh_coloured = colour_mesh(mesh)
    matrix = get_rotation_matrix(i, landmarks)
    mesh_rotated = transform_mesh(mesh_coloured, matrix)
    mesh_rotated.compute_vertex_normals()
    if i == 13: mesh_rotated.vertices = o3d.utility.Vector3dVector(flip_left(mesh_rotated))
    mesh_rotated.vertices = o3d.utility.Vector3dVector(center_mesh_vertices(mesh_rotated))
    meshes.append(mesh_rotated)
o3d.visualization.draw_geometries(meshes, mesh_show_back_face=True)

In [2]:
# visualise_canal_planes(range(1, 25), "lateral")
visualise_subject_planes(2) 

In [3]:
annotated = [3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 18, 20, 21, 22, 23, 24]
landmarks = ["right bony anterior", 'right top orbit', "nasion"]
visualise_canal_planes(annotated, "anterior", landmarks)
# visualise_canal_planes(annotated, "lateral")
matrices = []
for i in annotated:
    matrices.append(get_rotation_matrix(i, landmarks))
print(np.mean(matrices, axis = 0))
print(np.std(matrices, axis = 0))

[[ 0.28303564 -0.9222884   0.22235948]
 [ 0.89695403  0.1813116  -0.39342845]
 [ 0.32463933  0.31279246  0.88314247]]
[[0.07431068 0.04614838 0.11036052]
 [0.02780493 0.06835768 0.04865936]
 [0.06064983 0.10894328 0.03581236]]
